In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import xarray as xr
import numpy as np
from scipy import stats as st
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import cartopy
from netCDF4 import Dataset
from dateutil import parser
import cftime
import os
from scipy.stats import linregress
import regionmask
import csv
import matplotlib.colors as mcolors
import cftime
import datetime as dt
import math
%matplotlib inline

# Andmete sisselugemine ja ümbertöötlemine

In [ ]:
# Andmete sisselugemiseks

def loe_fail(kaust, alg, lopp, periood): # sesoon - periood: 'Q-NOV', aasta periood:'1Y'
    koos = []
    for item in os.listdir(kaust):
        path = os.path.join(kaust, item) 
        ds = xr.open_dataset(path).pr*86400 # teisendab ühikud mm-sse
        ds = ds.convert_calendar('standard', align_on = 'year')
        ds = ds.sel(time=slice(alg, lopp))
        ds = ds.resample(time=periood).sum()
        koos.append(ds)
        ds.close()
    return koos

In [ ]:
# Interpoleerib uutele koordinaatidele
def regrid1_CMIP6_to_ERA5(data):
    newlat = np.arange(57.4, 59.9, 0.25)
    newlon = np.arange(21.4, 28.4, 0.25)
    data2 = data.interp(lat=newlat, lon=newlon)   # interpolate to new coordinates
    data2 = data2.rename({'lat': 'latitude', 'lon': 'longitude'})
    
    return data2

In [ ]:
# Mask, mis eraldab andmepunktid, mis jäävad Eesti maismaapiiridesse
countries = regionmask.defined_regions.natural_earth_v5_0_0.countries_50
est_ind = countries.abbrevs.index('EST')
def masked_models(ssp):

    est_mask = countries.mask(ssp['longitude'], ssp['latitude'])
    maskedd = (ssp.where(est_mask == est_ind))
    #masked_data.append(masked)
    #maskedd = None
    est_mask = None

    return maskedd

In [ ]:
# Grupeerib andmed aastaaegade kaupa
def sesoonid(ssp, periood): #'MAM', 'JJA', 'SON', 'DJF'
    sesoon = []
    for i in range(len(ssp)):
        sesoon.append(ssp[i].sel(time=ssp[i]['time.season']==periood))
    return sesoon

## CMIP6 andmed

In [ ]:
# Päevased andmed 2070-2099
ssp126 = loe_fail('C:/Users/maria/loputoo/tulevik/126_yhis', '2070-01-01', '2099-12-31', '1D')
ssp245 = loe_fail('C:/Users/maria/loputoo/tulevik/245_yhis', '2070-01-01', '2099-12-31', '1D')
ssp370 = loe_fail('C:/Users/maria/loputoo/tulevik/370_yhis', '2070-01-01', '2099-12-31', '1D')
ssp585 = loe_fail('C:/Users/maria/loputoo/tulevik/585_yhis', '2070-01-01', '2099-12-31', '1D')
#Päevased andmed 1985-2014
hist_koos = loe_fail('C:/Users/maria/loputoo/minevik/hist_koos_yhis', '1985-03-01', '2014-11-30', '1D')

In [ ]:
# Aastased andmed 2015-2099
ssp126 = loe_fail('C:/Users/maria/loputoo/tulevik/126_yhis', '2015-01-01', '2099-12-31', '1Y')
ssp245 = loe_fail('C:/Users/maria/loputoo/tulevik/245_yhis', '2015-01-01', '2099-12-31', '1Y')
ssp370 = loe_fail('C:/Users/maria/loputoo/tulevik/370_yhis', '2015-01-01', '2099-12-31', '1Y')
ssp585 = loe_fail('C:/Users/maria/loputoo/tulevik/585_yhis', '2015-01-01', '2099-12-31', '1Y')
# Aastased andmed 1985-2014
hist_koos = loe_fail('C:/Users/maria/loputoo/minevik/hist_koos_yhis', '1985-01-01', '2014-12-31', '1Y')

In [ ]:
# Sesoonsed andmed 2015-2099
ssp126 = loe_fail('C:/Users/maria/loputoo/tulevik/126_yhis', '2015-03-01', '2099-11-30', 'Q-NOV')
ssp245 = loe_fail('C:/Users/maria/loputoo/tulevik/245_yhis', '2015-03-01', '2099-11-30', 'Q-NOV')
ssp370 = loe_fail('C:/Users/maria/loputoo/tulevik/370_yhis', '2015-03-01', '2099-11-30', 'Q-NOV')
ssp585 = loe_fail('C:/Users/maria/loputoo/tulevik/585_yhis', '2015-03-01', '2099-11-30', 'Q-NOV')
# Sesoonsed andmed 1985-2014
hist_koos = loe_fail('C:/Users/maria/loputoo/minevik/hist_koos_yhis', '1985-03-01', '2014-11-30', 'Q-NOV')

## ERA5 andmed

In [ ]:
# Võrsustab nulliga andmepunktid, kus sademeid on alla 1 mm
def alla1(ssp):
    for i in range(len(ssp)):
        ssp[i] =  ssp[i].dropna(dim = 'longitude', how = 'all')
        ssp[i] = ssp[i].dropna(dim = 'latitude', how = 'all')
        ssp[i] = np.where(np.less(ssp[i].values, 1), 0, ssp[i].values)

    return ssp

In [ ]:
era5 = xr.open_dataset('era5_1979_2014.nc').tp*1000

In [ ]:
# Liidab kokku ühe päeva sademete andmed.
era5_loik = era5_loik.resample(time='1D').sum()

In [ ]:
# Aastased andmed
era5_aasta = era5_loik.resample(time='1Y').sum()

## E-OBS andmed

In [ ]:
# Aastased andmed
eobs = xr.open_dataset('C:/Users/maria/loputoo/minevik/rr_ens_mean_0.25deg_reg_v29.0e.nc').sel(time=slice("1985-01-01", '2014-12-31'))
eobs = eobs.resample(time='1Y').sum().rr 
#Sesoonsed andmed
#eobs = xr.open_dataset('C:/Users/maria/loputoo/minevik/rr_ens_mean_0.25deg_reg_v29.0e.nc').sel(time=slice("1985-03-01", '2014-11-30'))
#eobs = eobs.resample(time='Q-NOV').sum().rr 

eobs = eobs.sel(latitude=slice(57, 60))
eobs = eobs.sel(longitude=slice(21, 29))


# CMIP6, ERA5 ja E-OBS ajaloolise perioodi andmete võrdlus mõõtmisandmetega

# Keskmine sademete hulk

In [ ]:
# Eraldab andmed huvipakkuvast perioodist, interpoleerib ja eraldab maski abil punktid, mis jäävad Eesti maismaapiiridesse
def loik(ssp, alg, lopp):
    periood = []
    for mudel in ssp:
        mudel_loik = mudel.sel(time=slice(str(alg), str(lopp)))
        mudel_loik = regrid1_CMIP6_to_ERA5(mudel_loik)
        mudel_loik =  masked_models(mudel_loik)
        periood.append(mudel_loik)
    return periood

In [ ]:
# Ansambli keskmise leidmiseks
def ssp_kesk(mudel):
    for el in mudel:
        kesk = np.nanmean(np.nanmean(mudel, axis = 0), axis = 0)
        kesk = xr.DataArray(kesk, 
            coords={'latitude': mudel[0].latitude,'longitude': mudel[0].longitude}, 
            dims=['latitude', 'longitude'])
    return kesk

## Keskmine aastane sademete hulk

In [ ]:
# Ajaloolise perioodi keskmised sademete andmed
hist_interp = loik(hist_koos, 1985, 2014)
hist_mask = ssp_kesk(hist_interp)
# Perioodi 2070-2099 keskmised sademete andmed
uus126 = loik(ssp126, 2070, 2099)
uus126 = ssp_kesk(uus126)
uus245 = loik(ssp245, 2070, 2099)
uus245 = ssp_kesk(uus245)
uus370 = loik(ssp370, 2070, 2099)
uus370 = ssp_kesk(uus370)
uus585 = loik(ssp585, 2070, 2099)
uus585 = ssp_kesk(uus585)
# Muutused võrreldes ajaloolise perioodiga stsenaariumite kaupa
hist_126 = uus126 - hist_mask
hist_245 = uus245 - hist_mask
hist_370 = uus370 - hist_mask
hist_585 = uus585 - hist_mask

### Graafikud

In [ ]:
# Muutused on ühisel värviskaalal
def graafik_aasta(ds1, ds2, ds3, ds4, ds5, joonise_nimi):
    pealkiri = ['SSP1-2.6', 'SSP2-4.5', 'SSP3-7.0', 'SSP5-8.5', '1985-2014']
    ds = [ds1, ds2, ds3, ds4, ds5]
    hist = ds5
    ssp =  np.concatenate([ds1, ds2, ds3, ds4])
    ala = [21, 29, 57, 60] 

    # Leiab maksimaalse ja minimaalse väärtuse kõigi stsenaarimite jaoks.
    hist_min = np.nanmin(hist)
    hist_max = np.nanmax(hist)

    ssp_min = np.nanmin(ssp)
    ssp_max = np.nanmax(ssp)
    
    fig, axes = plt.subplots(1, 5, figsize=(28, 6), constrained_layout = True, subplot_kw={'projection': ccrs.Mercator()})

    for i, ax in enumerate(axes.flatten()): 
        ax.add_feature(cfeature.BORDERS, linestyle=':')
        ax.add_feature(cfeature.LAKES, alpha=0.25)
        ax.coastlines()
        ax.set_extent(ala, crs=ccrs.PlateCarree())
        ax.set_title(pealkiri[i], fontsize = 20)
        
        if i == 4:
            kaart_hist = ds[i].plot(ax=ax, transform=ccrs.PlateCarree(), cmap='Greens', vmin=hist_min, vmax=hist_max, add_colorbar = False)

        else:
            kaart = ds[i].plot(ax=ax, transform=ccrs.PlateCarree(), cmap='Paired', vmin=ssp_min, vmax=ssp_max, add_colorbar = False)

        if i in [0, 1, 2, 3, 4]:
            ax.set_title(pealkiri[i], fontsize = 30)
            if i == 4:
                cbar2 = plt.colorbar(kaart_hist, ax=ax, shrink = 0.75)
                ticklabs2 = cbar2.ax.get_yticklabels()
                cbar2.ax.set_yticklabels(ticklabs2, fontsize = 20) # muudab värviskaala numbrite suurust
                cbar2.set_label('Sademeid aasta kohta (mm)', rotation=90, fontsize = 20)
            if i == 3:
                cbar1 = plt.colorbar(kaart, ax=ax, shrink = 0.75)
                ticklabs1 = cbar1.ax.get_yticklabels()
                cbar1.ax.set_yticklabels(ticklabs1, fontsize = 20) # muudab värviskaala numbrite suurust
                cbar1.set_label('Sademete muutus (mm)', rotation=90, fontsize = 20)




    plt.savefig('C:/Users/maria/loputoo/Joonised/kesk_aasta_ruumiline/' + joonise_nimi + '.png', bbox_inches='tight')
    plt.show()

In [ ]:
# Tulevikliimat kirjeldavatel graafikutel omaette värviskaala
def graafik_aasta(ds1, ds2, ds3, ds4, ds5, joonise_nimi):
    pealkiri = ['SSP1-2.6', 'SSP2-4.5', 'SSP3-7.0', 'SSP5-8.5', '1985-2014']
    ds = [ds1, ds2, ds3, ds4, ds5]
    ssp =  np.concatenate([ds1, ds2, ds3, ds4])
    ala = [21, 29, 57, 60] 
    
    fig, axes = plt.subplots(1, 5, figsize=(28, 6), constrained_layout = True, subplot_kw={'projection': ccrs.Mercator()})

    for i, ax in enumerate(axes.flatten()): 
        ax.add_feature(cfeature.BORDERS, linestyle=':')
        ax.add_feature(cfeature.LAKES, alpha=0.25)
        ax.coastlines()
        ax.set_extent(ala, crs=ccrs.PlateCarree())
        
        
        if i == 4:
            kaart_hist = ds[i].plot(ax=ax, transform=ccrs.PlateCarree(), cmap='Greens',  vmin=np.nanmin(ds[i]), vmax=np.nanmax(ds[i]), add_colorbar = False)
            cbar = plt.colorbar(kaart_hist, ax=ax, shrink = 0.75)
            ticklabs = cbar.ax.get_yticklabels()
            cbar.ax.set_yticklabels(ticklabs, fontsize = 20)
            cbar.set_label('Sademeid aastas (mm)', rotation=90, fontsize = 20)
        else:
            kaart = ds[i].plot(ax=ax, transform=ccrs.PlateCarree(), cmap='Reds', vmin=np.nanmin(ds[i]), vmax=np.nanmax(ds[i]), add_colorbar = False)
            cbar2 = plt.colorbar(kaart, ax=ax, shrink = 0.75)
            ticklabs2 = cbar2.ax.get_yticklabels()
            cbar2.ax.set_yticklabels(ticklabs2, fontsize = 20) # muudab värviskaala numbrite suurust
        if i in range(0, 5):
            ax.set_title(pealkiri[i], fontsize = 20)
            if  i == 3:
                cbar2.set_label('Sademete muutus (mm)', rotation=90, fontsize = 20)

    plt.savefig('C:/Users/maria/loputoo/Joonised/kesk_aasta_ruumiline/' + joonise_nimi + '.png', bbox_inches='tight')
    plt.show()

## Keskmine sademete hulk aastaagade kaupa

In [ ]:
# Ajaloolise perioodi sademete andmed
kevad_hist = sesoonid(hist_koos, 'MAM')
suvi_hist = sesoonid(hist_koos, 'JJA')
sygis_hist = sesoonid(hist_koos, 'SON')
talv_hist = sesoonid(hist_koos, 'DJF')

hist_kevad = loik(kevad_hist, 1985, 2014)
hist_suvi = loik(suvi_hist, 1985, 2014)
hist_sygis = loik(sygis_hist, 1985, 2014)
hist_talv = loik(talv_hist, 1985, 2014)
hist_kevad = ssp_kesk(hist_kevad)
hist_suvi = ssp_kesk(hist_suvi)
hist_sygis = ssp_kesk(hist_sygis)
hist_talv = ssp_kesk(hist_talv)
# Perioodi 2070-2099 sademete andmed kliimastsenaariumite kaupa
kevad_126 = sesoonid(ssp126, 'MAM')
suvi_126 = sesoonid(ssp126, 'JJA')
sygis_126 = sesoonid(ssp126, 'SON')
talv_126 = sesoonid(ssp126, 'DJF')
kevad_126 = loik(kevad_126, 2070, 2099)
suvi_126 = loik(suvi_126, 2070, 2099)
sygis_126 = loik(sygis_126, 2070, 2099)
talv_126 = loik(talv_126, 2070, 2099)
kevad_126 = ssp_kesk(kevad_126)
suvi_126 = ssp_kesk(suvi_126)
sygis_126 = ssp_kesk(sygis_126)
talv_126 = ssp_kesk(talv_126)

kevad_245 = sesoonid(ssp245, 'MAM')
suvi_245 = sesoonid(ssp245, 'JJA')
sygis_245 = sesoonid(ssp245, 'SON')
talv_245 = sesoonid(ssp245, 'DJF')
kevad_245 = loik(kevad_245, 2070, 2099)
suvi_245 = loik(suvi_245, 2070, 2099)
sygis_245 = loik(sygis_245, 2070, 2099)
talv_245 = loik(talv_245, 2070, 2099)
kevad_245 = ssp_kesk(kevad_245)
suvi_245 = ssp_kesk(suvi_245)
sygis_245 = ssp_kesk(sygis_245)
talv_245 = ssp_kesk(talv_245)

kevad_370 = sesoonid(ssp370, 'MAM')
suvi_370 = sesoonid(ssp370, 'JJA')
sygis_370 = sesoonid(ssp370, 'SON')
talv_370 = sesoonid(ssp370, 'DJF')
kevad_370 = loik(kevad_370, 2070, 2099)
suvi_370 = loik(suvi_370, 2070, 2099)
sygis_370 = loik(sygis_370, 2070, 2099)
talv_370 = loik(talv_370, 2070, 2099)
kevad_370 = ssp_kesk(kevad_370)
suvi_370 = ssp_kesk(suvi_370)
sygis_370 = ssp_kesk(sygis_370)
talv_370 = ssp_kesk(talv_370)

kevad_585 = sesoonid(ssp585, 'MAM')
suvi_585 = sesoonid(ssp585, 'JJA')
sygis_585 = sesoonid(ssp585, 'SON')
talv_585 = sesoonid(ssp585, 'DJF')
kevad_585 = loik(kevad_585, 2070, 2099)
suvi_585 = loik(suvi_585, 2070, 2099)
sygis_585 = loik(sygis_585, 2070, 2099)
talv_585 = loik(talv_585, 2070, 2099)
kevad_585 = ssp_kesk(kevad_585)
suvi_585 = ssp_kesk(suvi_585)
sygis_585 = ssp_kesk(sygis_585)
talv_585 = ssp_kesk(talv_585)

#Muutused võrrelds ajaloolise periooidga
hist_126_kevad = kevad_126 - hist_kevad
hist_126_suvi = suvi_126 - hist_suvi
hist_126_sygis = sygis_126 - hist_sygis
hist_126_talv = talv_126 - hist_talv

hist_245_kevad = kevad_245 - hist_kevad
hist_245_suvi = suvi_245 - hist_suvi
hist_245_sygis = sygis_245 - hist_sygis
hist_245_talv = talv_245 - hist_talv

hist_370_kevad = kevad_370 - hist_kevad
hist_370_suvi = suvi_370 - hist_suvi
hist_370_sygis = sygis_370 - hist_sygis
hist_370_talv = talv_370 - hist_talv

hist_585_kevad = kevad_585 - hist_kevad
hist_585_suvi = suvi_585 - hist_suvi
hist_585_sygis = sygis_585 - hist_sygis
hist_585_talv = talv_585 - hist_talv

### Graafikud

In [ ]:
# Kõik muutused ühisel värviskaalal
def graafik_sesoon(ds1, ds2, ds3, ds4, ds5, ds6, ds7, ds8, ds9, ds10, ds11, ds12, ds13, ds14, ds15, ds16, ds17, ds18, ds19, ds20, joonise_nimi):
    y_telg= ['Kevad', 'Suvi', 'Sügis', 'Talv']
    pealkiri = ['SSP1-2.6', 'SSP2-4.5', 'SSP3-7.0', 'SSP5-8.5', '1985-2014']
    ds = [ds1, ds2, ds3, ds4, ds5, ds6, ds7, ds8, ds9, ds10, ds11, ds12, ds13, ds14, ds15, ds16, ds17, ds18, ds19, ds20]
    hist = np.concatenate([ds[4], ds[9], ds[14], ds[19]])
    ssp =  np.concatenate([ds[0:4], ds[5:9], ds[10:14], ds[15:19]])
    ala = [21, 29, 57, 60]

    # Leiab maksimaalse ja minimaalse väärtuse kõigi stsenaarimite jaoks.
    hist_min = np.nanmin(hist)
    hist_max = np.nanmax(hist)

    ssp_min = np.nanmin(ssp)
    ssp_max = np.nanmax(ssp)

    fig, axes = plt.subplots(4, 5, figsize=(30, 15), layout="constrained", subplot_kw={'projection': ccrs.Mercator()})
    

    for i, ax in enumerate(axes.flatten()):
        ax.add_feature(cfeature.BORDERS, linestyle=':')
        ax.add_feature(cfeature.LAKES, alpha=0.25)
        ax.coastlines()
        ax.set_extent(ala, crs=ccrs.PlateCarree())

        if (i+1) % 5 == 0:
            kaart_hist = ds[i].plot(ax=ax, transform=ccrs.PlateCarree(), cmap='Greens', vmin=hist_min, vmax=hist_max, add_colorbar = False)
        else:
            norm = mcolors.TwoSlopeNorm(vmin=-ssp_max, vcenter=0, vmax=ssp_max) # seob 0 väärtuse vastavusse valge värviga kaardil
            kaart = ds[i].plot(ax=ax, transform=ccrs.PlateCarree(), norm=norm, cmap='RdBu_r', vmin=-ssp_max, vmax=ssp_max, add_colorbar = False)

        if i in [0, 1, 2, 3, 4]:
            ax.set_title(pealkiri[i], fontsize = 30)#
            if i == 4:
                cbar2 = plt.colorbar(kaart_hist, ax=axes[0:, ][:, 4])
                ticklabs2 = cbar2.ax.get_yticklabels()
                cbar2.ax.set_yticklabels(ticklabs2, fontsize = 30) # muudab värviskaala numbrite suurust
                cbar2.set_label('Sademeid aastaaja kohta (mm)', rotation=90, fontsize = 30)
            if i == 3:
                cbar1 = plt.colorbar(kaart, ax=axes[0:, ][:, 3])
                cbar1.ax.set_yscale('linear')
                ticklabs1 = cbar1.ax.get_yticklabels()
                cbar1.ax.set_yticklabels(ticklabs1, fontsize = 30) # muudab värviskaala numbrite suurust
                cbar1.set_label('Sademete muutus aastaaja kohta (mm)', rotation=90, fontsize = 30)
        if i in [0, 5, 10, 15, 20]:
            ind = int(i/5)
            ax.annotate(y_telg[ind], xy=(-0.1, 0.15), xycoords="axes fraction", rotation=90,fontsize = 30)

    plt.savefig('C:/Users/maria/loputoo/Joonised/kesk_aasta_ruumiline/' + joonise_nimi + '.png',  bbox_inches='tight')
    plt.show()

In [ ]:
# Iga tulevikliimat kirjeldava graafiku jaoks on loodud oma värviskaala
def graafik_sesoon(ds1, ds2, ds3, ds4, ds5, ds6, ds7, ds8, ds9, ds10, ds11, ds12, ds13, ds14, ds15, ds16, ds17, ds18, ds19, ds20, joonise_nimi):
    y_telg= ['Kevad', 'Suvi', 'Sügis', 'Talv']
    pealkiri = ['SSP1-2.6', 'SSP2-4.5', 'SSP3-7.0', 'SSP5-8.5', '1985-2014']
    ds = [ds1, ds2, ds3, ds4, ds5, ds6, ds7, ds8, ds9, ds10, ds11, ds12, ds13, ds14, ds15, ds16, ds17, ds18, ds19, ds20]
    hist = np.concatenate([ds[4], ds[9], ds[14], ds[19]])
    ssp =  np.concatenate([ds[0:4], ds[5:9], ds[10:14], ds[15:19]])
    ala = [21, 29, 57, 60]

    # Leiab maksimaalse ja minimaalse väärtuse kõigi stsenaarimite jaoks.
    hist_min = np.nanmin(hist)
    hist_max = np.nanmax(hist)


    fig, axes = plt.subplots(4, 5, figsize=(30, 15), layout="constrained", subplot_kw={'projection': ccrs.Mercator()})
  
    for i, ax in enumerate(axes.flatten()):
        ax.add_feature(cfeature.BORDERS, linestyle=':')
        ax.add_feature(cfeature.LAKES, alpha=0.25)
        ax.coastlines()
        ax.set_extent(ala, crs=ccrs.PlateCarree())

        if i in [4, 9, 14, 19]:
            kaart_hist = ds[i].plot(ax=ax, transform=ccrs.PlateCarree(), cmap='Greens', vmin=hist_min, vmax=hist_max, add_colorbar = False)
            
        if i == 6:
            norm1 = mcolors.TwoSlopeNorm(vmin=np.nanmin(ds[i]), vcenter=0, vmax=-np.nanmin(ds[i])) # seob 0 väärtuse vastavusse valge värviga kaardil
            kaart1 = ds[i].plot(ax=ax, transform=ccrs.PlateCarree(), norm=norm1, cmap='RdBu_r', vmin=np.nanmin(ds[i]), vmax=-np.nanmin(ds[i]), add_colorbar = False)
            cbar1 = plt.colorbar(kaart1)
            cbar1.ax.set_yscale('linear')
            
            ticklabs1 = cbar1.ax.get_yticklabels()
            float_tick_labels = [float(label.get_text().replace('−', '-')) for label in ticklabs1]
            integer_tick_labels = [int(label) for label in float_tick_labels if label.is_integer()]
            cbar1.ax.set_yticklabels(integer_tick_labels, fontsize = 20) # muudab värviskaala numbrite suurust
        if i in range(7, 9):
            print(np.nanmin(ds[i]), np.nanmax(ds[i]))
            kaart2 = ds[i].plot(ax=ax, transform=ccrs.PlateCarree(), cmap='Blues_r', vmin=np.nanmin(ds[i]), vmax=np.nanmax(ds[i]), add_colorbar = False)
            cbar2 = plt.colorbar(kaart2)
            ticklabs2 = cbar2.ax.get_yticklabels()
            
            float_tick_labels2 = [float(label.get_text().replace('−', '-')) for label in ticklabs2]
            integer_tick_labels2 = [int(label) for label in float_tick_labels2 if label.is_integer()]
            integer_tick_labels2 = [int(label) for label in integer_tick_labels2 if label in range(math.floor(np.nanmin(ds[i])), math.ceil(np.nanmax(ds[i])))]
            
            cbar2.ax.set_yticks(integer_tick_labels2)
            cbar2.ax.set_yticklabels(integer_tick_labels2, fontsize=20)
 
        if i in [0, 1, 2, 3, 5, 10, 11, 12, 13, 15, 16, 17, 18]:
            kaart3 = ds[i].plot(ax=ax, transform=ccrs.PlateCarree(), cmap='Reds', vmin=np.nanmin(ds[i]), vmax=np.nanmax(ds[i]), add_colorbar = False)
            cbar3 = plt.colorbar(kaart3)
            cbar3.ax.set_yscale('linear')
            ticklabs3 = cbar3.ax.get_yticklabels()
            float_tick_labels3 = [float(label.get_text().replace('−', '-')) for label in ticklabs3]
            integer_tick_labels3 = [int(label) for label in float_tick_labels3 if label.is_integer()]
            integer_tick_labels3 = [int(label) for label in integer_tick_labels3 if label in range(math.floor(np.nanmin(ds[i])), math.ceil(np.nanmax(ds[i])))]
            
            cbar3.ax.set_yticks(integer_tick_labels3)
            cbar3.ax.set_yticklabels(integer_tick_labels3, fontsize=20)
        if i in [0, 1, 2, 3, 4]:
            ax.set_title(pealkiri[i], fontsize = 30)
            if i == 4:
                cbar_hist = plt.colorbar(kaart_hist, ax=axes[0:, ][:, 4], aspect = 20)
                ticklabs_hist = cbar_hist.ax.get_yticklabels()
                cbar_hist.ax.set_yticklabels(ticklabs_hist, fontsize = 20) # muudab värviskaala numbrite suurust
                cbar_hist.set_label('Sademeid aastaaja kohta (mm)', rotation=90, fontsize = 20)
            ind = int(i/5)
            ax.annotate(y_telg[ind], xy=(-0.1, 0.15), xycoords="axes fraction", rotation=90,fontsize = 30)

    plt.savefig('C:/Users/maria/loputoo/Joonised/kesk_aasta_ruumiline/' + joonise_nimi + '.png',  bbox_inches='tight')
    plt.show()

# Järjestikused sademete vabad päevad

In [2]:
# Leiab järjestikuste sademete vabade päevade arvu iga Eesti ruumipunkti kohta
# Keskmistab üle 30 aasta ning kõigi ansambli liikmete
# Tagastab DataArray formaadis andmed
def jarjestikused_sademete_vabad(ssp):
    jarjest_koos = []
    for el in ssp:
        regridded_mudel = regrid1_CMIP6_to_ERA5(el)
        masked = masked_models(regridded_mudel)
        kuivad = (masked < 0.1)+0
        max_jarjest = xr.zeros_like(masked.isel(time=0))
        
        for aasta in aasta_70_99:
            for lat_ind, lati in enumerate(masked['latitude']):
                for lon_ind, loni in enumerate(masked['longitude']):
                    kuivad_aastas =kuivad.sel(time = str(aasta), latitude=lati, longitude=loni)
                    vahed = np.diff(np.concatenate(([0], kuivad_aastas, [0])))
                    alg = np.where(vahed == 1)[0]
                    lopp = np.where(vahed == -1)[0]
                    jarjestikused = lopp - alg
        
                    if len(jarjestikused) == 0:
                        max_jarjest[lat_ind, lon_ind] = 0
                    else:
                        max_jarjest[lat_ind, lon_ind] = np.max(jarjestikused)
                        
        jarjest_koos.append(max_jarjest)
        kesk = xr.DataArray(data=np.mean(jarjest_koos, axis = 0),
             dims=list(masked.dims[1:]),
             coords=dict(
                 lon= masked.longitude,
                 lat=masked.latitude,
             ),
            name = 'pr'
        )
    return(kesk)

In [ ]:
# Pikimad kuivaperioodid aastas kliimastsenaariumite kaupa
kesk_126 = jarjestikused_sademete_vabad(ssp126)
kesk_245 = jarjestikused_sademete_vabad(ssp245)
kesk_370 = jarjestikused_sademete_vabad(ssp370)
kesk_585 = jarjestikused_sademete_vabad(ssp585)
kesk_hist = arjestikused_sademete_vabad(hist_koos)
# Muutus võrreldes ajaloolise periooidga
hist_126 = kesk_126 - kesk_hist
hist_245 = kesk_245 - kesk_hist
hist_370 = kesk_370 - kesk_hist
hist_585 = kesk_585 - kesk_hist

In [ ]:
# Pikimad kuivaperioodid aastaaja kohta kliimastsenaariumite kaupa
kesk_126_kevad = jarjestikused_sademete_vabad(kevad_126)
kesk_126_suvi = jarjestikused_sademete_vabad(suvi_126)
kesk_126_sygis = jarjestikused_sademete_vabad(sygis_126)
kesk_126_talv = jarjestikused_sademete_vabad(talv_126)

kesk_245_kevad = jarjestikused_sademete_vabad(kevad_245)
kesk_245_suvi = jarjestikused_sademete_vabad(suvi_245)
kesk_245_sygis = jarjestikused_sademete_vabad(sygis_245)
kesk_245_talv = jarjestikused_sademete_vabad(talv_245)

kesk_370_kevad = jarjestikused_sademete_vabad(kevad_370)
kesk_370_suvi = jarjestikused_sademete_vabad(suvi_370)
kesk_370_sygis = jarjestikused_sademete_vabad(sygis_370)
kesk_370_talv = jarjestikused_sademete_vabad(talv_370)

kesk_585_kevad = jarjestikused_sademete_vabad(kevad_585)
kesk_585_suvi = jarjestikused_sademete_vabad(suvi_585)
kesk_585_sygis = jarjestikused_sademete_vabad(sygis_585)
kesk_585_talv = jarjestikused_sademete_vabad(talv_585)

kesk_hist_kevad= jarjestikused_sademete_vabad(kevad_hist)
kesk_hist_suvi = jarjestikused_sademete_vabad(suvi_hist)
kesk_hist_sygis = jarjestikused_sademete_vabad(sygis_hist)
kesk_hist_talv = jarjestikused_sademete_vabad(talv_hist)

# Muutus võrreldes ajaloolise periooidga
kevad_hist_126 =  kesk_126_kevad - kesk_hist_kevad 
suvi_hist_126 =  kesk_126_suvi - kesk_hist_suvi
sygis_hist_126 = kesk_126_sygis - kesk_hist_sygis
talv_hist_126 = kesk_126_talv - kesk_hist_talv

kevad_hist_245 =  kesk_245_kevad - kesk_hist_kevad
suvi_hist_245 =  kesk_245_suvi - kesk_hist_suvi
sygis_hist_245 = kesk_245_sygis - kesk_hist_sygis
talv_hist_245 =  kesk_245_talv - kesk_hist_talv

kevad_hist_370 = kesk_370_kevad - kesk_hist_kevad
suvi_hist_370 = kesk_370_suvi - kesk_hist_suvi 
sygis_hist_370 = kesk_370_sygis - kesk_hist_sygis 
talv_hist_370 = kesk_370_talv - kesk_hist_talv 

kevad_hist_585 = kesk_585_kevad - kesk_hist_kevad
suvi_hist_585 = kesk_585_suvi - kesk_hist_suvi
sygis_hist_585 = kesk_585_sygis - kesk_hist_sygis
talv_hist_585 = kesk_585_talv - kesk_hist_talv

## Graafikud.

In [ ]:
#Kuivaperioodid aastas
def graafik(ds1, ds2, ds3, ds4, ds5, joonise_nimi):
    pealkiri = ['SSP1-2.6', 'SSP2-4.5', 'SSP3-7.0', 'SSP5-8.5', '1985-2014']
    ds = [ds1, ds2, ds3, ds4, ds5]
    
    hist = []
    ssp = []

    for i, el in enumerate(ds):
        el = xr.where(el == 0, np.nan, el)
        if i == 4:
            hist.append(el)
        else:
            ssp.append(el)
        
    ala = [21, 29, 57, 60] 

    # Leiab maksimaalse ja minimaalse väärtuse kõigi stsenaarimite jaoks.
    hist_min = np.nanmin(hist)
    hist_max = np.nanmax(hist)

    ssp_min = np.nanmin(ssp)
    ssp_max = np.nanmax(ssp)
    
    fig, axes = plt.subplots(1, 5, figsize=(28, 6), constrained_layout = True, subplot_kw={'projection': ccrs.Mercator()})
    #fig.suptitle('2070-2099 sademete normi muutus võrreldes perioodiga 1985-2014', fontsize = 40)

    for i, ax in enumerate(axes.flatten()): 
        ds[i] = xr.where(ds[i] == 0, np.nan, ds[i])
        ax.add_feature(cfeature.BORDERS, linestyle=':')
        ax.add_feature(cfeature.LAKES, alpha=0.25)
        ax.coastlines()
        ax.set_extent(ala, crs=ccrs.PlateCarree())
        ax.set_title(pealkiri[i], fontsize = 20)
        
        if i == 4:
            kaart_hist = ds[i].plot(ax=ax, transform=ccrs.PlateCarree(), cmap='Greens', vmin=hist_min, vmax=hist_max, add_colorbar = False)

        else:
            kaart = ds[i].plot(ax=ax, transform=ccrs.PlateCarree(), cmap='Reds',  vmin=ssp_min, vmax=ssp_max, add_colorbar = False)

        if i in [0, 1, 2, 3, 4]:
            ax.set_title(pealkiri[i], fontsize = 30)
            if i == 4:
                cbar2 = plt.colorbar(kaart_hist, ax=ax, shrink = 0.75)
                ticklabs2 = cbar2.ax.get_yticklabels()
                float_tick_labels2 = [float(label.get_text().replace('−', '-')) for label in ticklabs2]
                round_tick_labels2 = [np.round(label, 2) for label in float_tick_labels2]
                cbar2.ax.set_yticklabels(round_tick_labels2, fontsize = 20)
                cbar2.set_label('Päevade arv aastas', rotation=90, fontsize = 20)
            if i == 3:
                cbar1 = plt.colorbar(kaart, ax=ax, shrink = 0.75)
                ticklabs1 = cbar1.ax.get_yticklabels()
                float_tick_labels = [float(label.get_text().replace('−', '-')) for label in ticklabs1]
                round_tick_labels = [np.round(label, 2) for label in float_tick_labels]
                print(ticklabs1, float_tick_labels)
                cbar1.ax.set_yticklabels(round_tick_labels, fontsize = 20)
                cbar1.set_label('Muutus päevade arvus', rotation=90, fontsize = 20)
       
    plt.savefig('C:/Users/maria/loputoo/Joonised/yle5/' + joonise_nimi + '.png',  bbox_inches='tight')
    plt.show()

In [ ]:
# Kuivaperioodid aastaaegade kaupa
def graafik(ds1, ds2, ds3, ds4, ds5, ds6, ds7, ds8, ds9, ds10, ds11, ds12, ds13, ds14, ds15, ds16, ds17, ds18, ds19, ds20, joonise_nimi):
    y_telg= ['Kevad', 'Suvi', 'Sügis', 'Talv']
    pealkiri = ['SSP1-2.6', 'SSP2-4.5', 'SSP3-7.0', 'SSP5-8.5', '1985-2014']
    ds = [ds1, ds2, ds3, ds4, ds5, ds6, ds7, ds8, ds9, ds10, ds11, ds12, ds13, ds14, ds15, ds16, ds17, ds18, ds19, ds20]
    hist1 = np.concatenate([ds[4], ds[9], ds[14], ds[19]])
    ssp1 =  np.concatenate([ds[0:4], ds[5:9], ds[10:14], ds[15:19]])

    hist = []
    ssp = []

    for el in hist1:
        el = xr.where(el == 0, np.nan, el)
        hist.append(el)
    for el in ssp1:
        el = xr.where(el == 0, np.nan, el)
        ssp.append(el)
    ala = [21, 29, 57, 60] 
    
    # Leiab maksimaalse ja minimaalse väärtuse kõigi stsenaarimite jaoks.
    hist_min = np.nanmin(hist)
    hist_max = np.nanmax(hist)

    ssp_min = np.nanmin(ssp)
    ssp_max = np.nanmax(ssp)
    
    fig, axes = plt.subplots(4, 5, figsize=(30, 15), layout="constrained", subplot_kw={'projection': ccrs.Mercator()})

    for i, ax in enumerate(axes.flatten()): 
        ds[i] = xr.where(ds[i] == 0, np.nan, ds[i])
        ax.add_feature(cfeature.BORDERS, linestyle=':')
        ax.add_feature(cfeature.LAKES, alpha=0.25)
        ax.coastlines()
        ax.set_extent(ala, crs=ccrs.PlateCarree())

        if (i+1) % 5 == 0:
            kaart_hist = ds[i].plot(ax=ax, transform=ccrs.PlateCarree(), cmap='Greens', vmin=hist_min, vmax=hist_max, add_colorbar = False)
        else:
            #print(np.max(ds[i]), np.min(ds[i]))
            norm = mcolors.TwoSlopeNorm(vmin=-ssp_max, vcenter=0, vmax=ssp_max) # seab 0 väärtuse vastavusse valge värviga kaardil
            kaart = ds[i].plot(ax=ax, transform=ccrs.PlateCarree(), cmap='RdBu_r', norm = norm, vmin=-ssp_max, vmax=ssp_max, add_colorbar = False)
            
        if i in [0, 1, 2, 3, 4]:
            ax.set_title(pealkiri[i], fontsize = 30)
            if i == 4:
                cbar2 = plt.colorbar(kaart_hist, ax=axes[0:, ][:, 4])
                ticklabs2 = cbar2.ax.get_yticklabels()
                cbar2.ax.set_yticklabels(ticklabs2, fontsize = 30) # muudab värviskaala numbrite suurust
                cbar2.set_label('Päevade arv aastaaja kohta', rotation=90, fontsize = 30)
            if i == 3:
                cbar1 = plt.colorbar(kaart, ax=axes[0:, ][:, 3])
                cbar1.ax.set_yscale('linear')
                ticklabs1 = cbar1.ax.get_yticklabels()
                cbar1.ax.set_yticklabels(ticklabs1, fontsize = 30) # muudab värviskaala numbrite suurust
                cbar1.set_label('Muutus päevade arvus', rotation=90, fontsize = 30)
        if i in [0, 5, 10, 15, 20]:
            ind = int(i/5)
            ax.annotate(y_telg[ind], xy=(-0.1, 0.15), xycoords="axes fraction", rotation=90,fontsize = 30)

    plt.savefig('C:/Users/maria/loputoo/Joonised/yle5/' + joonise_nimi + '.png', bbox_inches='tight')
    plt.show()

# Suure sajuga päevad

In [ ]:
# Leiab ühe Eesti ruumipunkti kohta, mitu päeva sajab seal üle 10 mm
def yle10(ssp):
    for i in range(len(ssp)):
        ssp[i] = regrid1_CMIP6_to_ERA5(ssp[i])
        ssp[i]  = masked_models(ssp[i])
        ssp[i] =  ssp[i].dropna(dim = 'longitude', how = 'all')
        ssp[i] = ssp[i].dropna(dim = 'latitude', how = 'all')
        for j in range(len(ssp[i])):
            ssp[i][j] = np.where(np.less(ssp[i][j].values, 10), np.nan, ssp[i][j].values)
            ssp[i][j] = np.where(np.greater_equal(ssp[i][j].values, 10), 1, ssp[i][j].values)
    return ssp

In [ ]:
# Graafiliseks kujutamiseks on kasutatud sama koodi, mis kuivaperioodide puhulgi